# **Reinforcement Learning: An Introduction** - Chapter 9
Thomas Hopkins

## **Exercise 9.1**
I do not think that the eligibility trace method would perform as well as the model planning method. This is because the trace for each state would fall off for the states that were many steps away from the goal. Furthermore, some of the states involved in the eligibility trace update would point to the state that followed it most recently. This problem does not occur in the model planning method because each transition encountered is stored as part of the model and randomly sampled during simulated learning.

## **Exercise 9.2**
The DynaQ+ agent likely performed better in both phases due to its exploration. It prioritized its exploration based on its model of the environment while the others explored randomly.

## **Exercise 9.3**
The difference between DynaQ+ and DynaQ narrowed because the DynaQ+ agent spends some of its computation exploring states that have not been encountered in a while. This allows the other DynaQ agent to "catch up" in cumulative reward by not exploring nearly as much.

## **Exercise 9.4**
It seems like the DynaQ+ agent with the exploration mechanism performed during action selection performs better than the mechanism that alters rewards.

In [137]:
using Base.Iterators
using Random
using Plots
gr()

STATES = collect(product(1:6, 1:9))
GRID = zeros(6, 9)
GRID[4, 2:9] .= 1
ACTIONS = [(0, -1), (-1, 0), (0, 1), (1, 0)] # (left, up, right, down)
START = (6, 4)
GOAL = (1, 9)
    

function select_action(Q_values, state, n_vals; kappa = 0.1, method = false, greedy = false)
    if !greedy
        if rand() < 0.1
            action = rand(ACTIONS)
            return findfirst(x -> all(x .== action), ACTIONS), action
        end
    end
    values = Q_values[state...]
    if method
        values .+= (kappa * sqrt.(n_vals[state...]))
    end
    a = argmax(values)
    return a, ACTIONS[a]
end

function step(state, action)
    new_state = [state[1] + action[1], state[2] + action[2]]
    if new_state[1] <= 0
        new_state[1] = 1
    elseif new_state[1] > 6
        new_state[1] = 6
    end
    if new_state[2] <= 0
        new_state[2] = 1
    elseif new_state[2] > 9
        new_state[2] = 9
    end
    if GRID[new_state...] == 1
        new_state = state
    else
        new_state = (new_state[1], new_state[2])
    end
    reward = 0.0
    terminated = false
    if new_state == GOAL
        reward = 1.0
        terminated = true
    end
    return reward, new_state, terminated
end

function dynaq_plus(Q_values, model, n_vals; n=5, num_time_steps = 50000, kappa = 0.5, alpha = 0.1, gamma = 0.95, method = false)
    cumulative_reward = 0.0
    state = START
    a, action = select_action(Q_values, state, n_vals; kappa = kappa, method = method, greedy = false)
    terminated = false
    for e = 1:num_time_steps
        a, action = select_action(Q_values, state, n_vals; kappa = kappa, method = method, greedy = false)
        reward, new_state, terminated = step(state, action)
        na, new_action = select_action(Q_values, new_state, n_vals; greedy = true)
        if !method
            Q_values[state...][a] += alpha * ((reward + kappa * sqrt.(n_vals[state...][a])) + gamma * Q_values[new_state...][na] - Q_values[state...][a])
        else
            Q_values[state...][a] += alpha * (reward + gamma * Q_values[new_state...][na] - Q_values[state...][a])
        end
        for i = 1:6, j = 1:9
            n_vals[i, j] .+= 1
        end
        n_vals[state...][a] = 0
        model[state...][a] = (new_state, reward)
        state = new_state
        if terminated
            state = START
        end
        cumulative_reward += reward
        for i = 1:n
            S = findall(x -> any([!ismissing(x[i]) for i = 1:4]), model)
            s = rand(S)
            A = findall(x -> !ismissing(x), model[s])
            a = rand(A)
            sp, r = model[s][a]
            ap, _ = select_action(Q_values, sp, n_vals; greedy = true)
            Q_values[s][a] += alpha * (reward + gamma * Q_values[sp...][ap] - Q_values[s][a])
        end
    end
    return Q_values, cumulative_reward
end

dynaq_plus (generic function with 3 methods)

In [138]:
q_values = [[0.0 for a in ACTIONS] for i = 1:6, j = 1:9]
model = [Any[missing for a in ACTIONS] for i = 1:6, j = 1:9]
n_vals = [[0, 0, 0, 0] for i = 1:6, j = 1:9]
Q, reward_original = dynaq_plus(q_values, model, n_vals)

([[17.04668001194853, 17.112144798924962, 18.183839071035756, 17.680571550212512] [17.05680452976106, 18.251288132229085, 19.673205335867912, 18.848569942891046] … [0.0, 0.0, 0.0, 0.0] [0.0, 0.0, 0.0, 0.0]; [17.62706195519849, 17.097717715797128, 19.043217637726272, 17.754445637428045] [17.721813245203823, 17.88857767031447, 20.43074727026122, 18.695405340047767] … [18.554805277684135, 0.0, 0.0, 0.0] [0.0, 0.0, 0.0, 0.0]; … ; [16.496064306291835, 17.063876045394597, 15.834178562978641, 15.87403683422662] [16.45509899899592, 15.953579727633313, 15.457256724639448, 15.346112316512855] … [12.997948983920647, 12.45500518827363, 11.912014102098347, 11.940747391556904] [12.358036841680798, 0.0, 0.0, 0.0]; [15.97848437102069, 16.55119000909181, 15.45968936026643, 15.972502041430257] [15.963520503637955, 15.972325262126615, 14.822553931684038, 15.421811825057414] … [12.382676646065885, 12.419725584130235, 11.523268744527625, 12.038255541029246] [11.952089961904017, 11.88620721740693, 0.0, 11.5

In [139]:
q_values = [[0.0 for a in ACTIONS] for i = 1:6, j = 1:9]
model = [Any[missing for a in ACTIONS] for i = 1:6, j = 1:9]
n_vals = [[0, 0, 0, 0] for i = 1:6, j = 1:9]
Q, reward_new = dynaq_plus(q_values, model, n_vals; method = true)

([[605.0298209986272, 605.3467846460104, 575.3426952295258, 635.5793597757557] [604.7316509499418, 575.3174646013858, 546.6665554141003, 604.9234572978479] … [451.5653192399962, 433.89089971122, 1.7493640022644337e-9, 447.64837249492007] [0.0, 0.0, 0.0, 0.0]; [636.2610092579076, 604.8055618773469, 604.6934931088492, 667.2147955040028] [635.1922103535043, 575.4519327856597, 574.4559636062952, 634.901907831363] … [468.31844382016175, 431.9696464826603, 426.3868292499176, 468.08117369537774] [446.433456447583, 1.1527725762606654e-7, 427.45910824984463, 446.4906147416382]; … ; [761.4160740929423, 754.8397761192113, 768.8041934214067, 766.6382256571759] [773.3103016503813, 767.9723714121699, 779.3773388596063, 766.7809956862867] … [701.2744223139007, 672.40960911291, 643.3043842893677, 670.5323125226782] [672.1204920648514, 641.9606197846043, 642.2973142211533, 636.7442401803871]; [749.6928523406486, 759.1738999205086, 758.1130630254507, 749.1436377057173] [771.5042787885719, 769.6697365349

In [140]:
reward_original

0.0

In [141]:
reward_new

13.0